In [19]:
from braket.circuits import Circuit
import numpy as np
from braket.aws import AwsDevice, AwsQuantumTask
from braket.circuits import Circuit
from braket.devices import LocalSimulator 
from braket.tracking import Tracker

In [20]:
class TwoBitCircuit():
    def __init__(self, code, control, act):
        self.__name__ = code
        self.control = control
        self.act = act
        self.circuit = Circuit()
        self._activate_bell_state()
        if code == 'a0b0':
            self._apply_a0b0()
        elif code == 'a0b1':
            self._apply_a0b1()
        elif code == 'a1b0':
            self._apply_a1b0()
        elif code == 'a1b1':
            self._apply_a1b1()
        else:
            raise ValueError('Wrong code.')

    def _activate_bell_state(self):
        self.circuit = Circuit()
        self.circuit.h(self.control).cnot(self.control, self.act)
        return

    def _apply_a0b0(self):
        self.circuit.s(self.act).h(self.act).t(self.act).h(self.act)
        return

    def _apply_a0b1(self):
        self.circuit.s(self.act).h(self.act).ti(self.act).h(self.act)
        return

    def _apply_a1b0(self):
        self.circuit.h(self.control)
        self.circuit.s(self.act).h(self.act).t(self.act).h(self.act)
        return

    def _apply_a1b1(self):
        self.circuit.h(self.control)
        self.circuit.s(self.act).h(self.act).ti(self.act).h(self.act)
        return

In [21]:
device = AwsDevice("arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1")

shotsnum = 50
cq = 0
aq = 1

a0b0_circuit = TwoBitCircuit('a0b0',cq,aq).circuit
a0b1_circuit = TwoBitCircuit('a0b1',cq,aq).circuit
a1b0_circuit = TwoBitCircuit('a1b0',cq,aq).circuit
a1b1_circuit = TwoBitCircuit('a1b1',cq,aq).circuit

with Tracker() as tracker:
    a0b0_task = device.run(a0b0_circuit, shots=shotsnum)
    a0b1_task = device.run(a0b1_circuit, shots=shotsnum)
    a1b0_task = device.run(a1b0_circuit, shots=shotsnum)
    a1b1_task = device.run(a1b1_circuit, shots=shotsnum)

In [22]:
print(tracker.quantum_tasks_statistics())
print(tracker.simulator_tasks_cost())
print(tracker.qpu_tasks_cost())
print(a0b0_task.id)
print(a0b1_task.id)
print(a1b0_task.id)
print(a1b1_task.id)
print(a0b0_task)

{'arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1': {'shots': 200, 'tasks': {'CREATED': 4}}}
0
7.2000000000
arn:aws:braket:us-east-1:443313522425:quantum-task/5cd6a051-76c2-46e3-b8c1-66116a2f61b2
arn:aws:braket:us-east-1:443313522425:quantum-task/f09a4ae0-dc14-421e-83d9-b96dcd8cba16
arn:aws:braket:us-east-1:443313522425:quantum-task/0a2c6a11-085c-44a5-99c0-4e8a30018573
arn:aws:braket:us-east-1:443313522425:quantum-task/e461e8b8-6313-42a2-b806-bd7ddf58a41c
AwsQuantumTask('id/taskArn':'arn:aws:braket:us-east-1:443313522425:quantum-task/5cd6a051-76c2-46e3-b8c1-66116a2f61b2')


In [23]:
a0b0_result = a0b0_task.result()
a0b1_result = a0b1_task.result()
a1b0_result = a1b0_task.result()
a1b1_result = a1b1_task.result()

In [24]:
a0b0_measurements = a0b0_result.measurements
a0b1_measurements = a0b1_result.measurements
a1b0_measurements = a1b0_result.measurements
a1b1_measurements = a1b1_result.measurements

In [26]:
np.savetxt('aria1_a0b0_raw.txt',a0b0_measurements,fmt="%i")
np.savetxt('aria1_a0b1_raw.txt',a0b1_measurements,fmt="%i")
np.savetxt('aria1_a1b0_raw.txt',a1b0_measurements,fmt="%i")
np.savetxt('aria1_a1b1_raw.txt',a1b1_measurements,fmt="%i")

In [53]:
# import raw data
a0b0_raw = np.loadtxt("aria1_a0b0_raw.txt", dtype = int)
a0b1_raw = np.loadtxt("aria1_a0b1_raw.txt", dtype = int)
a1b0_raw = np.loadtxt("aria1_a1b0_raw.txt", dtype = int)
a1b1_raw = np.loadtxt("aria1_a1b1_raw.txt", dtype = int)

print(len(a0b0_raw))
print(len(a0b1_raw))
print(len(a1b0_raw))
print(len(a1b1_raw))

# random basis selection in Alice and Bob
finalResult = np.array([], dtype = int)
trial = 0
rng = np.random.default_rng()
while len(a0b0_raw)>0 and len(a0b1_raw)>0 and len(a1b0_raw)>0 and len(a1b1_raw)>0:
    trial = trial + 1
    randomBasis = rng.integers(2, size = 2)
    if (randomBasis == [0, 0]).all():
        value, a0b0_raw = a0b0_raw[0], a0b0_raw[1:]
    elif (randomBasis == [0, 1]).all():
        value, a0b1_raw = a0b1_raw[0], a0b1_raw[1:]
    elif (randomBasis == [1, 0]).all():
        value, a1b0_raw = a1b0_raw[0], a1b0_raw[1:]
    elif (randomBasis == [1, 1]).all():
        value, a1b1_raw = a1b1_raw[0], a1b1_raw[1:]
    else:
        pass
    finalResult = np.append(finalResult, value)
print(trial)
print(finalResult[1])

50
50
50
50
144
0


In [54]:
print(len(finalResult))
print(len(a0b0_raw))
print(len(a0b1_raw))
print(len(a1b0_raw))
print(len(a1b1_raw))
finalResult = np.concatenate([finalResult, a0b0_raw.flatten(), a0b1_raw.flatten(), a1b0_raw.flatten(), a1b1_raw.flatten()])
print(len(finalResult))

288
15
19
22
0
400


In [55]:
def calculate_min_entropy(sequence):
    sequence = np.asarray(sequence, dtype=float)  # Convert sequence to float
    p = np.mean(sequence)  # Proportion of ones
    max_prob = max(p, 1 - p)
    if max_prob == 0:  # Handle the case where all bits are the same
        return 0
    min_entropy = -np.log2(max_prob)
    return min_entropy


minentropy = calculate_min_entropy(finalResult)
print(minentropy)
extractnum = np.round(200 * minentropy).astype(int)
print(extractnum)

0.9364970576938421
187


In [56]:
from scipy.linalg import toeplitz, matmul_toeplitz
mc = rng.integers(2, size = extractnum)
mr = rng.integers(2, size = len(finalResult))
result = np.remainder(matmul_toeplitz((mc, mr), finalResult), 2).astype('int8')
print(len(result))

187


In [57]:
np.savetxt('aria1_DIQRNG.txt', result, fmt="%i")